In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort, FullSort
import numpy as np
import matplotlib.pyplot as plt
import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.models import Model
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from deel.lip.model import Sequential
from deel.lip.activations import GroupSort, FullSort
from tensorflow.keras.constraints import max_norm
from Autoencoderclasses import Encoder, Decoder, DenseEncoder, DenseDecoder


gamma = 4.0
beta_T = 50.0
beta_U = 2.0
Delta = 0.8
h_c = 0.002
L = np.pi
N_s = 45
N_integration = int(Delta/h_c)
delta_z = L/N_s

np.random.seed(123)

2023-05-21 09:04:04.694887: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 09:04:05.174863: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-21 09:04:05.228713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 09:04:05.228755: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
import json
with open('data.json') as json_file:
    data_dict = json.load(json_file)

from pickle import load
scaler_X = load(open('scaler_X.pkl', 'rb'))
scaler_y = load(open('scaler_y.pkl', 'rb'))

In [3]:
dir_list = ['Dense 300 units 6 rd 1.0 noise.h5', 'SpectralDense 300 units 6 rd 1.0 noise.h5']

for title in dir_list:
    if title[0] == 'S':
        encoder = tf.keras.models.load_model( title , custom_objects={'Encoder' : Encoder, 'Decoder' : Decoder})
    else:
        encoder = tf.keras.models.load_model(title , custom_objects={'DenseEncoder' : DenseEncoder, 'DenseDecoder' : DenseDecoder})
    np.array(encoder.layers[0].call(np.ones([1,46])))
    RNN_input = np.zeros([4*len(data_dict.keys()), int(N_integration / 4 / 10) , 8])
    RNN_output = np.zeros([4*len(data_dict.keys()),int(N_integration / 4 / 10) , 6])
    count = 0
    for key in list(data_dict.keys()):
        encoded_simulation = np.zeros([len(data_dict.keys()), 41, 6])
        simulation = scaler_X.transform(np.array(data_dict[key]))
        parameters = key.split(',')
        u1 = float(parameters[1])/100
        u2 = float(parameters[2])/100
        for i in range(0,len(simulation),10):
            encoded_simulation[count, i//10 ,:] = np.array(encoder.layers[0].call(np.array([simulation[i,:]])))
        initial_state = encoded_simulation[count, 0 ,:]
        trajectory =  encoded_simulation[count, 1:41 ,:]
        RNN_output[4*count:4*count+4,:,:] = np.split(trajectory,4)

        input1 =  np.concatenate([encoded_simulation[count, 0 ,:],np.array([u1,u2])]).reshape([1,8]).repeat(10,axis=0)
        RNN_input[4*count,:,:] = input1
        input2 =  np.concatenate([encoded_simulation[count, 10 ,:],np.array([u1,u2])]).reshape([1,8]).repeat(10,axis=0)
        RNN_input[4*count+1,:,:] = input2
        input3 =  np.concatenate([encoded_simulation[count, 20 ,:],np.array([u1,u2])]).reshape([1,8]).repeat(10,axis=0)
        RNN_input[4*count+2,:,:] = input3    
        input4 =  np.concatenate([encoded_simulation[count, 30 ,:],np.array([u1,u2])]).reshape([1,8]).repeat(10,axis=0)
        RNN_input[4*count+3,:,:] = input4
        count = count + 1
        print(count/len(data_dict.keys())*100)
    with open(title + ' RNN_input.npy', 'wb') as f:
        np.save(f,RNN_input)
    with open(title + ' RNN_output.npy', 'wb') as f:
        np.save(f,RNN_output)

    X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.3, random_state=123)
    print(title)
    checkpoint = ModelCheckpoint(filepath = "RNN for " + title, monitor="val_loss" ,verbose=0, save_best_only=True,mode="min")
    model = Sequential()
    model.add(SimpleRNN(64, activation='tanh', return_sequences=True))
    model.add(SimpleRNN(64, activation='tanh', return_sequences=True))
    model.add(Dense(6, activation='linear'))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4), loss='mean_squared_error')
    history = model.fit(X_train, y_train, epochs=6000, batch_size=256, validation_split=0.25, verbose=0 , callbacks = [checkpoint])
            


2023-05-21 09:09:44.209884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-21 09:09:44.210180: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-21 09:09:44.211048: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-21 09:09:44.211087: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-21 09:09:44.238066: W tensorflow/c

0.010797969981643452
0.021595939963286903
0.032393909944930355
0.043191879926573806
0.05398984990821725
0.06478781988986071
0.07558578987150416
0.08638375985314761
0.09718172983479105
0.1079796998164345
0.11877766979807797
0.12957563977972142
0.14037360976136484
0.15117157974300832
0.16196954972465177
0.17276751970629523
0.18356548968793868
0.1943634596695821
0.20516142965122555
0.215959399632869
0.22675736961451248
0.23755533959615593
0.24835330957779936
0.25915127955944284
0.2699492495410863
0.2807472195227297
0.2915451895043732
0.30234315948601664
0.3131411294676601
0.32393909944930355
0.334737069430947
0.34553503941259045
0.35633300939423385
0.36713097937587735
0.3779289493575208
0.3887269193391642
0.39952488932080765
0.4103228593024511
0.4211208292840946
0.431918799265738
0.4427167692473815
0.45351473922902497
0.46431270921066836
0.47511067919231187
0.4859086491739553
0.4967066191555987
0.5075045891372423
0.5183025591188857
0.5291005291005291
0.5398984990821726
0.550696469063816
0